In [4]:
%pip install CatBoost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    return summary

In [9]:
df = pd.read_csv("/content/Untitled spreadsheet - Sheet1.csv")
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,5849.0,0.0,0.0,360.0,Yes,1
1,4583.0,1508.0,128.0,360.0,Yes,0
2,3000.0,0.0,66.0,360.0,Yes,1
3,2583.0,2358.0,120.0,360.0,Yes,1
4,6000.0,0.0,141.0,360.0,Yes,1


In [10]:
print ("Total number of rows in dataset = {}".format(df.shape[0]))
print ("Total number of columns in dataset = {}".format(df.shape[1]))

Total number of rows in dataset = 614
Total number of columns in dataset = 6


In [11]:
result = resumetable(df)
result

Dataset Shape: (614, 6)


,Name,dtypes,Missing,Uniques,First Value,Second Value
0,ApplicantIncome,float64,2,503,5849.0,4583.0
1,CoapplicantIncome,float64,2,287,0.0,1508.0
2,LoanAmount,float64,3,203,0.0,128.0
3,Loan_Amount_Term,float64,2,11,360.0,360.0
4,Credit_History,object,0,2,Yes,Yes
5,Loan_Status,int64,0,2,1,0


In [12]:
target_col = "Loan_Status"
X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,              # separate the x and y in training and testing dataset
                                                    test_size=0.33, 
                                                    random_state=42)


In [14]:
X_train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
244,3406.0,4417.0,123.0,360.0,Yes
393,1993.0,1625.0,113.0,180.0,Yes
310,2917.0,0.0,84.0,360.0,Yes
408,8300.0,0.0,152.0,300.0,No
572,16666.0,0.0,275.0,360.0,Yes


In [15]:
features = list(X_train.columns)


In [16]:
cat_features = ["Credit_History"]


In [17]:
model_cb = CatBoostClassifier(task_type='GPU', iterations=100,  # fitting the model
                              random_state = 2021, 
                              eval_metric="F1")


In [18]:
model_cb.fit(X_train, y_train, cat_features= cat_features, plot=True, 
             eval_set=(X_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.199227
0:	learn: 0.8571429	test: 0.7885305	best: 0.7885305 (0)	total: 6.23ms	remaining: 617ms
1:	learn: 0.8553259	test: 0.8275862	best: 0.8275862 (1)	total: 10.5ms	remaining: 517ms
2:	learn: 0.8562300	test: 0.8251748	best: 0.8275862 (1)	total: 15.4ms	remaining: 498ms
3:	learn: 0.8576000	test: 0.8309859	best: 0.8309859 (3)	total: 20.1ms	remaining: 483ms
4:	learn: 0.8557692	test: 0.8309859	best: 0.8309859 (3)	total: 25.1ms	remaining: 478ms
5:	learn: 0.8539326	test: 0.8339223	best: 0.8339223 (5)	total: 29ms	remaining: 454ms
6:	learn: 0.8520900	test: 0.8309859	best: 0.8339223 (5)	total: 33.8ms	remaining: 449ms
7:	learn: 0.8502415	test: 0.8398577	best: 0.8398577 (7)	total: 37.7ms	remaining: 434ms
8:	learn: 0.8520900	test: 0.8309859	best: 0.8398577 (7)	total: 42.8ms	remaining: 432ms
9:	learn: 0.8520900	test: 0.8368794	best: 0.8398577 (7)	total: 45.7ms	remaining: 411ms
10:	learn: 0.8520900	test: 0.8309859	best: 0.8398577 (7)	total: 50.8ms	remaining: 411ms
11:	learn: 0.8

In [19]:
y_pred = model_cb.predict(X_test)

In [20]:
f1_score(y_test,y_pred)

0.8398576512455516